# Attention 메커니즘 구현하기

## 3.1 긴 시퀀스 모델링의 문제점
- Attention 이전의 문제점
    - RNN 이전 스텝의 출력이 현재 스텝의 입력으로 사용되는 신경망
    - 핵심 idea는 encoder가 전체 입력 텍스트를 하나의 은닉 상태로 처리
    - decoder가 이 은닉 상태를 받아 출력을 생성
    - RNN의 제약사항은 decoder가 encoder 이전의 은닉 상태를 참조할 수 없다는 것
    - **즉, 맥락을 놓칠 수 있다.**

## 3.2 Attention 메커니즘으로 데이터 의존성 포착하기
- Bahdanau attention은 RNN을 수정하여 decoding 단계마다 decoder가 선택적으로 입력 시퀀스의 서로 다른 부분을 참조할 수 있다.
- self Attention은 시퀀스의 표현을 계산할 때 입력 시퀀스에 있는 각 위치가 동일 시퀀스에 있는 다른 모든 위치와의 관련성을 고려하거나 주의를 기울일 수 있다.

## self Attention으로 입력의 서로 다른 부분에 주의 기울이기
- self는 하나의 입력 시퀀스에 있는 **서로 다른 위치의 원소 사이에 어텐션 가중치를 계산**하는 방식을 의미
- self Attention의 목표
    - 다른 모든 입력 원소의 정보를 조합하여
    - 각각의 입력 원소에 대한 문맥 벡터를 계산하는 것

### 3.3.1 훈련 가능한 가중치가 없는 간단한 self Attention 메커니즘
- "Your journey starts with one step"
- x는 특정 토큰을 표현하는 d 차원 임베딩 벡터다
    - "Your" [0.4][0.1][0.8] (예시는 3차원 벡터)
    - 문맥 벡터인 z는 x1 ~ xt(여기서 t는 상수) 사이의 정보를 담은 임베딩이다.
    - 문맥 벡터는 입력 시퀀스에 있는 모든 원소의 정보를 통합해 각 원소의 표현을 풍부하게 만드는 것이 목적이다.

다음은 3차원 벡터로 임베딩된 입력 시퀀스가 있다고 가정

In [1]:
import torch

inputs = torch.tensor(
    [[0.43, 0.15, 0.89],    # Your
     [0.55, 0.87, 0.66],    # journey
     [0.57, 0.85, 0.64],    # starts
     [0.22, 0.58, 0.33],    # with
     [0.77, 0.25, 0.10],    # one
     [0.05, 0.80, 0.55]]    # step
)

- self Attention 구현의 첫 단계는 attention score를 계산하는 것이다.
- attention score는 쿼리 x와 다른 모든 입력 토큰 사이의 점곱으로 결정한다.

In [2]:
query = inputs[1]

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


> [!note]
> - 점곱은 두 벡터를 원소끼리 곱한 다음 모두 더하는 방법이다.
> - 점곱은 하나의 유사도 척도로 볼 수 있다. (두 벡터가 얼마나 가까이 놓여 있는지 정량화할 수 있기 때문)
> - 점곱이 높을수록 두 원소 사이의 유사도와 attention score가 높다.

In [3]:
res = 0
for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


- attention score 정규화는 attention 가중치의 합이 1이 되도록 하기 위해서다.
- 정규화는 해석하기 용이하고 LLM 훈련 시 안정성을 유지하는 데 도움이 된다.

In [4]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("attention weights: ", attn_weights_2_tmp)
print("합: ", attn_weights_2_tmp.sum())

attention weights:  tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
합:  tensor(1.0000)


- attention score를 softmax로 정규화
- softmax는 attention weights 합이 1이다.

In [ ]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("attention weights: ", attn_weights_2)
print("합: ", attn_weights_2.sum())

- 임베딩된 입력 토큰 x와 각 토큰에 해당하는 attention 가중치를 곱한 후 모두 더해서 문맥 벡터 z 계산

In [11]:
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)

for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 모든 입력 토큰에 대해 attention 가중치 계산하기
- 모든 문맥 벡터를 계산하기 위해 코드 수정

In [15]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])
